<a id="introduction"></a>
## Introduction to cuDF
#### By Paul Hendricks
-------

In this notebook, we will show how to work with cuDF DataFrames in RAPIDS.

**Table of Contents**

* [Introduction to cuDF](#introduction)
* [Setup](#setup)
* [cuDF DataFrame Basics](#dataframes)
* [cuDF Series Basics](#series)
* [Input/Output](#io)
* [cuDF API](#cudfapi)
* [Conclusion](#conclusion)

<a id="setup"></a>
## Setup

This notebook was tested using the following Docker containers:

* `rapidsai/rapidsai-nightly:0.8-cuda10.0-devel-ubuntu18.04-gcc7-py3.7` from [DockerHub - rapidsai/rapidsai-nightly](https://hub.docker.com/r/rapidsai/rapidsai-nightly)

This notebook was run on the NVIDIA Tesla V100 GPU. Please be aware that your system may be different and you may need to modify the code or install packages to run the below examples. 

If you think you have found a bug or an error, please file an issue here: https://github.com/rapidsai/notebooks/issues

Before we begin, let's check out our hardware setup by running the `nvidia-smi` command.

In [1]:
!nvidia-smi

Tue Jun 11 09:16:47 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   35C    P0    43W / 300W |      0MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   35C    P0    41W / 300W |      0MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

Next, let's see what CUDA version we have:

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


<a id="dataframes"></a>
## cuDF DataFrame Basics

As we showed in the previous tutorial, cuDF DataFrames are a tabular structure of data that reside on the GPU. We interface with these cuDF DataFrames in the same way we interface with Pandas DataFrames that reside on the CPU - with a few deviations.

In the next several sections, we'll show how to create and manipulate cuDF DataFrames. For more information on using cuDF DataFrames, check out the documentation: https://rapidsai.github.io/projects/cudf/en/latest/index.html

#### Creating a cudf DataFrame using lists

There are several ways to create a cuDF DataFrame. The easiest of these is to instantiate an empty cuDF DataFrame and then use Python list objects or NumPy arrays to create columns. Below, we import the cuDF library and create an empty cuDF DataFrame.

In [3]:
import cudf; print('cuDF Version:', cudf.__version__)


df = cudf.DataFrame()
print(df)

cuDF Version: 0.7.2+0.g3ebd286.dirty
Empty DataFrame
Columns: []
Index: []


Next, we can create two columns named `key` and `value` by using the bracket notation with the cuDF DataFrame and storing either a list of Python values or a NumPy array into that column.

In [4]:
import numpy as np; print('NumPy Version:', np.__version__)


# here we create two columns named "key" and "value"
df['key'] = [0, 1, 2, 3, 4]
df['value'] = np.arange(10, 15)
print(df)

NumPy Version: 1.16.2
   key  value
0    0     10
1    1     11
2    2     12
3    3     13
4    4     14


#### Creating a cudf DataFrame using a list of tuples or a dictionary

Another way we can create a cuDF DataFrame is by providing a mapping of column names to column values, either via a list of tuples or by using a dictionary. In the below examples, we create a list of two-value tuples; the first value is the name of the column - for example, `id` or `timestamp` - and the second value is a list of Python objects or Numpy arrays. Note that we don't have to constrain the data stored in our cuDF DataFrames to common data types like integers or floats - we can use more exotic data types such as datetimes or strings. We'll investigate how such data types behave on the GPU a bit later.

In [5]:
from datetime import datetime, timedelta


ids = np.arange(5)
t0 = datetime.strptime('2018-10-07 12:00:00', '%Y-%m-%d %H:%M:%S')
timestamps = [(t0+ timedelta(seconds=x)) for x in range(5)]
timestamps_np = np.array(timestamps, dtype='datetime64')

In [6]:
df = cudf.DataFrame([('id', ids), ('timestamp', timestamps_np)])
print(df)

   id               timestamp
0   0 2018-10-07T12:00:00.000
1   1 2018-10-07T12:00:01.000
2   2 2018-10-07T12:00:02.000
3   3 2018-10-07T12:00:03.000
4   4 2018-10-07T12:00:04.000


Alternatively, we can create a dictonary of key-value pairs, where each key in the dictionary represents a column name and each value associated with the key represents the values that belong in that column.

In [7]:
df = cudf.DataFrame({'id': ids, 'timestamp': timestamps_np})
print(df)

   id               timestamp
0   0 2018-10-07T12:00:00.000
1   1 2018-10-07T12:00:01.000
2   2 2018-10-07T12:00:02.000
3   3 2018-10-07T12:00:03.000
4   4 2018-10-07T12:00:04.000


#### Creating a cudf DataFrame from a Pandas DataFrame

Pandas DataFrames are a first class citizen within cuDF - this means that we can create a cuDF DataFrame from a Pandas DataFrame and vice versa.

In [8]:
import pandas as pd; print('Pandas Version:', pd.__version__)


pandas_df = pd.DataFrame({'a': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'b': [0.0, 0.1, 0.2, None, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]})
print(pandas_df)

Pandas Version: 0.23.4
     a    b
0    0  0.0
1    1  0.1
2    2  0.2
3    3  NaN
4    4  0.4
5    5  0.5
6    6  0.6
7    7  0.7
8    8  0.8
9    9  0.9
10  10  1.0


We can use the `cudf.from_pandas` or `cudf.DataFrame.from_pandas` functions to create a cuDF DataFrame from a Pandas DataFrame.

In [9]:
df = cudf.from_pandas(pandas_df)
# df = cudf.DataFrame.from_pandas(pandas_df)  # alternative
print(df)

   a    b
0  0  0.0
1  1  0.1
2  2  0.2
3  3     
4  4  0.4
5  5  0.5
6  6  0.6
7  7  0.7
8  8  0.8
9  9  0.9
[1 more rows]


#### Creating a cuDF DataFrame from cuDF Series (coming soon!)

#### Inspecting a cuDF DataFrame

There are several ways to inspect a cuDF DataFrame. The first method is to enter the cuDF DataFrame directly into the REPL. This shows us information about the type of the object, and metadata such as the number of rows or columns.

In [10]:
df = cudf.DataFrame({'a': np.arange(0, 100), 'b': np.arange(100, 0, -1)})

In [11]:
df

<cudf.DataFrame ncols=2 nrows=100 >

A second way to inspect a cuDF DataFrame is to wrap the object in a Python `print` function. This results in showing the rows and columns of the dataframe.

In [12]:
print(df)

   a    b
0  0  100
1  1   99
2  2   98
3  3   97
4  4   96
5  5   95
6  6   94
7  7   93
8  8   92
9  9   91
[90 more rows]


For very large dataframes, we often want to see the first couple rows. We can use the `head` method of a cuDF DataFrame to view the first N rows.

In [13]:
print(df.head())

   a    b
0  0  100
1  1   99
2  2   98
3  3   97
4  4   96


#### Columns

cuDF DataFrames store metadata such as information about columns or data types. We can access the columns of a cuDF DataFrame using the `.columns` attribute.

In [14]:
print(df.columns)

Index(['a', 'b'], dtype='object')


We can modify the columns of a cuDF DataFrame by modifying the `columns` attribute. We can do this by setting that attribute equal to a list of strings representing the new columns.

In [15]:
df.columns = ['c', 'd']
print(df.columns)

Index(['c', 'd'], dtype='object')


#### Data Types

We can also inspect the data types of the columns of a cuDF DataFrame using the `dtypes` attribute.

In [16]:
print(df.dtypes)

c    int64
d    int64
dtype: object


We can modify the data types of the columns of a cuDF DataFrame by passing in a cuDF Series with a modified data type. Be warned that silent errors may be introduced from nonsensical type conversations - for example, changing a float to an integer or vice versa.

In [17]:
df['c'] = df['c'].astype(np.float32)
df['d'] = df['d'].astype(np.int32)
print(df.dtypes)

c    float32
d      int32
dtype: object


#### Series

cuDF DataFrames are composed of rows and columns. Each column is represented using an object of type `cudf.dataframe.series.Series`. For example, if we subset a cuDF DataFrame using just one column we will be returned an object of type `cudf.dataframe.series.Series`.

In [18]:
print(type(df['c']))
print(df['c'])

<class 'cudf.dataframe.series.Series'>
0    0.0
1    1.0
2    2.0
3    3.0
4    4.0
5    5.0
6    6.0
7    7.0
8    8.0
9    9.0
[90 more rows]
Name: c, dtype: float32


#### Index


In [19]:
df.index

RangeIndex(start=0, stop=100)

#### Converting a cudf DataFrame to a Pandas DataFrame

We can convert a cuDF DataFrame back to a Pandas DataFrame using the `to_pandas` method.

In [20]:
pandas_df = df.to_pandas()
print(type(pandas_df))

<class 'pandas.core.frame.DataFrame'>


#### Converting a cudf DataFrame to a NumPy Array

Often we want to work with NumPy arrays. We can convert a cuDF DataFrame to a NumPy array by first converting it to a Pandas DataFrame using the `to_pandas` method followed by accessing the `values` attribute of the Pandas DataFrame.

In [21]:
numpy_array = df.to_pandas().values
print(type(numpy_array))

<class 'numpy.ndarray'>


#### Converting a cudf DataFrame to Other Data Formats

We can also convert a cuDF DataFrame to other data formats. 

For more information, see the documentation: https://docs.rapids.ai/api/cudf/stable/

<a id="series"></a>
## cuDF Series Basics (coming soon!)

To be edited.

<a id="io"></a>
## Input/Output

Before we process data and use it in machine learning models, we need to be able to load it into memory and write it after we're done using it. There are several ways to do this using cuDF.

#### Writing and Loading CSV Files

At this time, there is no direct way to use to cuDF to write directly to CSV. However, we can conver the cuDF DataFrame to a Pandas DataFrame and then write it directly to a CSV.

In [22]:
df.to_pandas().to_csv('./dataset.csv', index=False)

Perhaps one of the most common ways to create cuDF DataFrames is by loading a table that is stored as a file on disk. cuDF provides a lot of functionality for reading in a variety of different data formats. Below, we show how easy it is to read in a CSV file:

In [23]:
df = cudf.read_csv('./dataset.csv')
print(df)

     c    d
0  0.0  100
1  1.0   99
2  2.0   98
3  3.0   97
4  4.0   96
5  5.0   95
6  6.0   94
7  7.0   93
8  8.0   92
9  9.0   91
[90 more rows]


CSV files come in many flavors and cuDF tries to be as flexible as possible, mirroring the Pandas API wherever possible. For more information on possible parameters for working with files, see the cuDF IO documentation: 

https://rapidsai.github.io/projects/cudf/en/latest/api.html#cudf.io.csv.read_csv

<a id="cudfapi"></a>
## cuDF API

The cuDF API is pleasantly simple and mirrors the Pandas API as closely as possible. In this section, we will explore the cuDF API and show how to perform common data manipulation operations.

#### Selecting Rows or Columns

We can select rows from a cuDF DataFrame using slicing syntax. 

In [24]:
df = cudf.DataFrame({'a': np.arange(0, 100).astype(np.float32), 
                     'b': np.arange(100, 0, -1).astype(np.float32)})

In [25]:
print(df[0:5])

     a      b
0  0.0  100.0
1  1.0   99.0
2  2.0   98.0
3  3.0   97.0
4  4.0   96.0


There are several ways to select a column from a cuDF DataFrame.

In [26]:
print(df['a'])
# print(df.a)  # alternative

0    0.0
1    1.0
2    2.0
3    3.0
4    4.0
5    5.0
6    6.0
7    7.0
8    8.0
9    9.0
[90 more rows]
Name: a, dtype: float32


We can also select multiple columns by passing in a list of column names.

In [27]:
print(df[['a', 'b']])

     a      b
0  0.0  100.0
1  1.0   99.0
2  2.0   98.0
3  3.0   97.0
4  4.0   96.0
5  5.0   95.0
6  6.0   94.0
7  7.0   93.0
8  8.0   92.0
9  9.0   91.0
[90 more rows]


We can select specific rows and columns using the slicing syntax as well as passing in a list of column names.

In [28]:
print(df.loc[0:5, ['a']])
# print(df.loc[0:5, ['a', 'b']])  # to select multiple columns, pass in multiple column names

     a
0  0.0
1  1.0
2  2.0
3  3.0
4  4.0
5  5.0


#### Dropping Columns or Rows (coming soon!)

In [29]:
df = cudf.DataFrame({'a': np.arange(0, 100).astype(np.float32), 
                     'b': np.arange(100, 0, -1).astype(np.float32), 
                     'c': np.arange(100, 200).astype(np.float32)})

In [30]:
df.drop_column('a')

In [31]:
print(df)

       b      c
0  100.0  100.0
1   99.0  101.0
2   98.0  102.0
3   97.0  103.0
4   96.0  104.0
5   95.0  105.0
6   94.0  106.0
7   93.0  107.0
8   92.0  108.0
9   91.0  109.0
[90 more rows]


In [32]:
df = cudf.DataFrame({'a': np.arange(0, 100).astype(np.float32), 
                     'b': np.arange(100, 0, -1).astype(np.float32), 
                     'c': np.arange(100, 200).astype(np.float32)})

In [33]:
new_df = df.drop('a')

In [34]:
print(new_df)

       b      c
0  100.0  100.0
1   99.0  101.0
2   98.0  102.0
3   97.0  103.0
4   96.0  104.0
5   95.0  105.0
6   94.0  106.0
7   93.0  107.0
8   92.0  108.0
9   91.0  109.0
[90 more rows]


In [35]:
df = cudf.DataFrame({'a': np.arange(0, 100).astype(np.float32), 
                     'b': np.arange(100, 0, -1).astype(np.float32), 
                     'c': np.arange(100, 200).astype(np.float32)})

In [36]:
new_df = df.drop(['a', 'b'])

In [37]:
print(new_df)

       c
0  100.0
1  101.0
2  102.0
3  103.0
4  104.0
5  105.0
6  106.0
7  107.0
8  108.0
9  109.0
[90 more rows]


In [38]:
df = df[0:50]

#### Defining New Columns (coming soon!)

In [39]:
df = cudf.DataFrame({'a': np.arange(0, 100).astype(np.float32), 
                     'b': np.arange(100, 0, -1).astype(np.float32), 
                     'c': np.arange(100, 200).astype(np.float32)})

In [40]:
df['d'] = np.arange(200, 300).astype(np.float32)

In [41]:
data = np.arange(300, 400).astype(np.float32)
df.add_column('e', data)

In [42]:
print(df)

     a      b      c      d      e
0  0.0  100.0  100.0  200.0  300.0
1  1.0   99.0  101.0  201.0  301.0
2  2.0   98.0  102.0  202.0  302.0
3  3.0   97.0  103.0  203.0  303.0
4  4.0   96.0  104.0  204.0  304.0
5  5.0   95.0  105.0  205.0  305.0
6  6.0   94.0  106.0  206.0  306.0
7  7.0   93.0  107.0  207.0  307.0
8  8.0   92.0  108.0  208.0  308.0
9  9.0   91.0  109.0  209.0  309.0
[90 more rows]


#### Missing Data (coming soon!)

In [43]:
df = cudf.DataFrame({'a': [0, None, 2, 3, 4, 5, 6, 7, 8, None, 10],
                     'b': [0.0, 0.1, 0.2, None, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 
                     'c': [0.0, 0.1, None, None, 0.4, 0.5, None, 0.7, 0.8, 0.9, 1.0]})
print(df)

   a    b    c
0  0  0.0  0.0
1     0.1  0.1
2  2  0.2     
3  3          
4  4  0.4  0.4
5  5  0.5  0.5
6  6  0.6     
7  7  0.7  0.7
8  8  0.8  0.8
9     0.9  0.9
[1 more rows]


In [44]:
df['c'] = df['c'].fillna(999)

In [45]:
print(df)

   a    b      c
0  0  0.0    0.0
1     0.1    0.1
2  2  0.2  999.0
3  3       999.0
4  4  0.4    0.4
5  5  0.5    0.5
6  6  0.6  999.0
7  7  0.7    0.7
8  8  0.8    0.8
9     0.9    0.9
[1 more rows]


In [46]:
new_df = df.fillna(-1)

In [47]:
print(new_df)

    a     b      c
0   0   0.0    0.0
1  -1   0.1    0.1
2   2   0.2  999.0
3   3  -1.0  999.0
4   4   0.4    0.4
5   5   0.5    0.5
6   6   0.6  999.0
7   7   0.7    0.7
8   8   0.8    0.8
9  -1   0.9    0.9
[1 more rows]


#### Boolean Indexing (coming soon!)

In [48]:
df = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                     'b': np.random.randint(2, size=100).astype(np.int32), 
                     'c': np.arange(0, 100).astype(np.int32), 
                     'd': np.arange(100, 0, -1).astype(np.int32)})

In [49]:
mask = df['a'] == 3
df[mask]

<cudf.DataFrame ncols=4 nrows=25 >

#### Sorting Data (coming soon!)

In [50]:
df = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                     'b': np.random.randint(2, size=100).astype(np.int32), 
                     'c': np.arange(0, 100).astype(np.int32), 
                     'd': np.arange(100, 0, -1).astype(np.int32)})
print(df.head())

   a  b  c    d
0  0  0  0  100
1  0  1  1   99
2  0  1  2   98
3  0  0  3   97
4  0  1  4   96


In [51]:
print(df.sort_values('d').head())

   a  b   c  d
99  3  0  99  1
98  3  0  98  2
97  3  1  97  3
96  3  0  96  4
95  3  0  95  5


In [52]:
print(df.sort_values('c', ascending=False).head())

   a  b   c  d
99  3  0  99  1
98  3  0  98  2
97  3  1  97  3
96  3  0  96  4
95  3  0  95  5


In [53]:
print(df.sort_values(['a', 'b']).head())

   a  b  c    d
0  0  0  0  100
3  0  0  3   97
5  0  0  5   95
6  0  0  6   94
8  0  0  8   92


In [54]:
print(df.sort_values(['a', 'b'], ascending=False).head())

   a  b   c   d
75  3  1  75  25
80  3  1  80  20
81  3  1  81  19
82  3  1  82  18
84  3  1  84  16


In [55]:
print(df.sort_values(['a', 'b'], ascending=[False, True]).head())

   a  b   c   d
76  3  0  76  24
77  3  0  77  23
78  3  0  78  22
79  3  0  79  21
83  3  0  83  17


#### Statistical Operations (coming soon!)

In [56]:
df = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                     'b': np.random.randint(2, size=100).astype(np.int32), 
                     'c': np.arange(0, 100).astype(np.int32), 
                     'd': np.arange(100, 0, -1).astype(np.int32)})

In [57]:
df['a'].sum()

150

In [58]:
print(df.sum())

a     150
b      45
c    4950
d    5050
dtype: int64


#### Applymap Operations (coming soon!)

In [59]:
df = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                     'b': np.random.randint(2, size=100).astype(np.int32), 
                     'c': np.arange(0, 100).astype(np.int32), 
                     'd': np.arange(100, 0, -1).astype(np.int32)})

In [60]:
def add_ten_to_x(x):
    return x + 10

print(df['c'].applymap(add_ten_to_x))

0    10
1    11
2    12
3    13
4    14
5    15
6    16
7    17
8    18
9    19
[90 more rows]
Name: c, dtype: int32


#### Histogramming (coming soon!)

In [61]:
df = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                     'b': np.random.randint(2, size=100).astype(np.int32), 
                     'c': np.arange(0, 100).astype(np.int32), 
                     'd': np.arange(100, 0, -1).astype(np.int32)})

In [62]:
result = df['a'].value_counts()
print(result)

0    25
1    25
2    25
3    25
dtype: int64


#### Concatenations (coming soon!)

In [63]:
df1 = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                      'b': np.random.randint(2, size=100).astype(np.int32), 
                      'c': np.arange(0, 100).astype(np.int32), 
                      'd': np.arange(100, 0, -1).astype(np.int32)})
df2 = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                      'b': np.random.randint(2, size=100).astype(np.int32), 
                      'c': np.arange(0, 100).astype(np.int32), 
                      'd': np.arange(100, 0, -1).astype(np.int32)})

In [64]:
df = cudf.concat([df1, df2], axis=0)
df

<cudf.DataFrame ncols=4 nrows=200 >

In [65]:
df1 = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                      'b': np.random.randint(2, size=100).astype(np.int32), 
                      'c': np.arange(0, 100).astype(np.int32), 
                      'd': np.arange(100, 0, -1).astype(np.int32)})
df2 = cudf.DataFrame({'e': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                      'f': np.random.randint(2, size=100).astype(np.int32), 
                      'g': np.arange(0, 100).astype(np.int32), 
                      'h': np.arange(100, 0, -1).astype(np.int32)})

In [66]:
df = cudf.concat([df1, df2], axis=1)
df

<cudf.DataFrame ncols=8 nrows=100 >

#### Joins (coming soon!)

In [67]:
df1 = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                      'b': np.random.randint(2, size=100).astype(np.int32), 
                      'c': np.arange(0, 100).astype(np.int32), 
                      'd': np.arange(100, 0, -1).astype(np.int32)})
df2 = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                      'b': np.random.randint(2, size=100).astype(np.int32), 
                      'e': np.arange(0, 100).astype(np.int32), 
                      'f': np.arange(100, 0, -1).astype(np.int32)})

In [68]:
df = df1.merge(df2, on=['a'])
print(df.head())

   a  b_x   c   d  b_y   e   f
0  1    0  32  68    0  25  75
1  1    1  33  67    0  25  75
2  1    1  34  66    0  25  75
3  1    1  35  65    0  25  75
4  1    1  36  64    0  25  75


In [69]:
df = df1.merge(df2, on=['a', 'b'])
print(df.head())

   a  b  c    d  e    f
0  0  0  0  100  1   99
1  0  0  1   99  1   99
2  0  0  2   98  1   99
3  0  1  3   97  0  100
4  0  0  4   96  1   99


In [70]:
df = cudf.merge(df1, df2, on=['a'])
print(df.head())

   a  b_x   c   d  b_y   e   f
0  2    1  64  36    1  50  50
1  2    1  65  35    1  50  50
2  2    1  66  34    1  50  50
3  2    0  67  33    1  50  50
4  2    1  68  32    1  50  50


In [71]:
df = cudf.merge(df1, df2, on=['a', 'b'])
print(df.head())

   a  b  c    d  e    f
0  0  0  0  100  1   99
1  0  0  1   99  1   99
2  0  0  2   98  1   99
3  0  1  3   97  0  100
4  0  0  4   96  1   99


#### Appends (coming soon!)

In [72]:
df1 = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                      'b': np.random.randint(2, size=100).astype(np.int32), 
                      'c': np.arange(0, 100).astype(np.int32), 
                      'd': np.arange(100, 0, -1).astype(np.int32)})
df2 = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                      'b': np.random.randint(2, size=100).astype(np.int32), 
                      'e': np.arange(0, 100).astype(np.int32), 
                      'f': np.arange(100, 0, -1).astype(np.int32)})

In [73]:
print(df.a.head().append(df.b.head()))

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    1
9    0
dtype: int32


#### Groupbys (coming soon!)

In [74]:
df = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                     'b': np.random.randint(2, size=100).astype(np.int32), 
                     'c': np.arange(0, 100).astype(np.int32), 
                     'd': np.arange(100, 0, -1).astype(np.int32)})
print(df.head())

   a  b  c    d
0  0  1  0  100
1  0  1  1   99
2  0  1  2   98
3  0  0  3   97
4  0  0  4   96


In [75]:
aggregation = df.groupby('a').sum()
print(aggregation)

    b     c     d
a
0  13   300  2200
1  15   925  1575
2  13  1550   950
3  15  2175   325


In [76]:
aggregation = df.groupby(['a', 'b']).sum().to_pandas()
print(aggregation)

        c     d
a b            
0 0   158  1042
  1   142  1158
1 0   365   635
  1   560   940
2 0   763   437
  1   787   513
3 0   873   127
  1  1302   198


#### One Hot Encoding (coming soon!)

In [77]:
categories = [0, 1, 2, 3]
df = cudf.DataFrame({'a': np.repeat(categories, 25).astype(np.int32), 
                     'b': np.arange(0, 100).astype(np.int32), 
                     'c': np.arange(100, 0, -1).astype(np.int32)})
print(df.head())

   a  b    c
0  0  0  100
1  0  1   99
2  0  2   98
3  0  3   97
4  0  4   96


In [78]:
result = df.one_hot_encoding('a', prefix='a_', cats=categories)
print(result.head())
print(result.tail())

   a  b    c  a__0  a__1  a__2  a__3
0  0  0  100   1.0   0.0   0.0   0.0
1  0  1   99   1.0   0.0   0.0   0.0
2  0  2   98   1.0   0.0   0.0   0.0
3  0  3   97   1.0   0.0   0.0   0.0
4  0  4   96   1.0   0.0   0.0   0.0
   a   b  c  a__0  a__1  a__2  a__3
95  3  95  5   0.0   0.0   0.0   1.0
96  3  96  4   0.0   0.0   0.0   1.0
97  3  97  3   0.0   0.0   0.0   1.0
98  3  98  2   0.0   0.0   0.0   1.0
99  3  99  1   0.0   0.0   0.0   1.0


#### Categoricals (coming soon!)

In [79]:
pandas_df = pd.DataFrame({'key': [1, 2, 3, 4, 5, 6], 
                          'value': ['a', 'b', 'b', 'a', 'a', 'e']})
pandas_df['value'] = pandas_df['value'].astype('category')

df = cudf.DataFrame.from_pandas(pandas_df)
print(df)

   key  value
0    1      a
1    2      b
2    3      b
3    4      a
4    5      a
5    6      e


In [80]:
print(df.value.cat.categories)

('a', 'b', 'e')


In [81]:
print(df.value.cat.codes)

0    0
1    1
2    1
3    0
4    0
5    2
dtype: int8


<a id="conclusion"></a>
## Conclusion

In this notebook, we showed how to work with cuDF DataFrames in RAPIDS.

To learn more about RAPIDS, be sure to check out: 

* [Open Source Website](http://rapids.ai)
* [GitHub](https://github.com/rapidsai/)
* [Press Release](https://nvidianews.nvidia.com/news/nvidia-introduces-rapids-open-source-gpu-acceleration-platform-for-large-scale-data-analytics-and-machine-learning)
* [NVIDIA Blog](https://blogs.nvidia.com/blog/2018/10/10/rapids-data-science-open-source-community/)
* [Developer Blog](https://devblogs.nvidia.com/gpu-accelerated-analytics-rapids/)
* [NVIDIA Data Science Webpage](https://www.nvidia.com/en-us/deep-learning-ai/solutions/data-science/)